In [1]:
from mt.definitions import DATASET_DIR
import pandas as pd
from mt.repo_to_pt import nx_to_pyg_graph
from mt.featurize_repo import get_tree_from_code
import networkx as nx
import json
import pickle
import torch

/home/fox/projects/general/learning-portfolio/university/masters/masters-thesis/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
PYTRACEBUGS_DIR = DATASET_DIR / "pytracebugs"
PROC_DIR = PYTRACEBUGS_DIR / "processed"
PROC_DIR.mkdir(exist_ok=True)
TRAIN_DIR = PROC_DIR / "train"
TRAIN_DIR.mkdir(exist_ok=True)

In [16]:
counter = 0
# df = pd.read_pickle(PYTRACEBUGS_DIR / "stable_dataset/stable_code_train.pickle")
# for row in df.iterrows():
#     code: str | bytes = row[1]["before_merge"]
#     ast = get_tree_from_code(code.encode() if isinstance(code, str) else code)
#     ast_json = nx.node_link_data(ast)
#     with open(TRAIN_DIR / f"{counter}.json", "w") as f:
#         json.dump({"code": code.decode() if isinstance(code, bytes) else code, "ast": ast_json}, f)
#     data, maps = nx_to_pyg_graph(ast)
#     with open(TRAIN_DIR / f"{counter}.pkl", "wb") as f:
#         pickle.dump(maps, f)
#     torch.save((data, 0), TRAIN_DIR / f"batch_{counter}.pt")

In [15]:
df = pd.read_pickle(PYTRACEBUGS_DIR / "buggy_dataset/bugfixes_train.pickle")
df

,after_merge,before_merge,filename,full_file_code_after_merge,full_file_code_before_merge,function_name,url,source code and errors,full_traceback,traceback_type,before_merge_without_docstrings,after_merge_without_docstrings,before_merge_docstrings,after_merge_docstrings,path_to_snippet_before_merge,path_to_snippet_after_merge
1790,"def plot(result_pickle_file_path, show, plot_s...","def plot(result_dict_file, show, plot_save_fil...",rqalpha/mod/rqalpha_mod_sys_analyser/__init__.py,# -*- coding: utf-8 -*-\n#\n# Copyright 2017 R...,# -*- coding: utf-8 -*-\n#\n# Copyright 2017 R...,plot,https://github.com/ricequant/rqalpha/issues/109,"[{'piece_type': 'other', 'piece_content': 'rqa...","Traceback (most recent call last):\nFile ""c:\p...",TypeError,"def plot(result_dict_file, show, plot_save_fil...","def plot(result_pickle_file_path, show, plot_s...",[[sys_analyser] draw result DataFrame],[[sys_analyser] draw result DataFrame],buggy_snippets_files/e93817735d3042d739fe86677...,buggy_snippets_files/e93817735d3042d739fe86677...
2433,"def stream_logs(self):\n """"""Stream ...","def stream_logs(self):\n """"""Stream ...",binderhub/build.py,"""""""\nContains build of a docker image from a g...","""""""\nContains build of a docker image from a g...",Build.stream_logs,https://github.com/jupyterhub/binderhub/issues...,"[{'piece_type': 'error message', 'piece_conten...",/ # jupyter-repo2docker https://github.com/yuv...,FileNotFoundError,def stream_logs(self):\n \n ...,def stream_logs(self):\n \n ...,[],[],buggy_snippets_files/8241189c4267b81254c9ed07a...,buggy_snippets_files/8241189c4267b81254c9ed07a...
26618,"def addRecentProjectFile(self, projectFile...","def addRecentProjectFile(self, projectFile...",meshroom/ui/app.py,import logging\nimport os\nimport argparse\n\n...,import logging\nimport os\nimport argparse\n\n...,MeshroomApp.addRecentProjectFile,https://github.com/alicevision/meshroom/issues...,"[{'piece_type': 'error message', 'piece_conten...","[2020-05-23 16:12:48,660][ERROR] Traceback (mo...",OSError,"def addRecentProjectFile(self, projectFile...","def addRecentProjectFile(self, projectFile...",[],[],buggy_snippets_files/faddf4c059bd32cc1cad1a1ea...,buggy_snippets_files/faddf4c059bd32cc1cad1a1ea...
26622,"def addSfmAugmentation(self, withMVS=False...","def addSfmAugmentation(self, withMVS=False...",meshroom/ui/reconstruction.py,import logging\nimport os\nfrom threading impo...,import logging\nimport os\nfrom threading impo...,Reconstruction.addSfmAugmentation,https://github.com/alicevision/meshroom/issues...,"[{'piece_type': 'error message', 'piece_conten...","Traceback (most recent call last):\nFile ""C:\U...",RuntimeError,"def addSfmAugmentation(self, withMVS=False...","def addSfmAugmentation(self, withMVS=False...",[],[],buggy_snippets_files/dffb9602005cbea45f7d0c6d2...,buggy_snippets_files/dffb9602005cbea45f7d0c6d2...
28217,def load_pymathics_doc(self):\n if ...,def load_pymathics_doc(self):\n if ...,mathics/doc/doc.py,#!/usr/bin/env python3\n# -*- coding: utf-8 -*...,#!/usr/bin/env python3\n# -*- coding: utf-8 -*...,MathicsMainDocumentation.load_pymathics_doc,https://github.com/mathics/Mathics/issues/906,"[{'piece_type': 'error message', 'piece_conten...",$ mathicsserver\nwarning: database file /home/...,KeyError,def load_pymathics_doc(self):\n if ...,def load_pymathics_doc(self):\n if ...,[],[],buggy_snippets_files/c98cf1a03e1d7e716b228fbe8...,buggy_snippets_files/c98cf1a03e1d7e716b228fbe8...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
625588,"def touch_member(self, data, ttl=None, per...","def touch_member(self, data, ttl=None, per...",patroni/dcs/consul.py,from __future__ import absolute_import\nimport...,from __future__ import absolute_import\nimport...,Consul.touch_member,https://github.com/zalando/patroni/issues/853,"[{'piece_type': 'error message', 'piece_conten...","2018-11-06 23:06:17,073 INFO: Lock owner: db1;...",urllib3.exceptions.ReadTimeoutError,"def touch_member(self, data, ttl=None, per

In [17]:
# TRAIN
df = pd.read_pickle(PYTRACEBUGS_DIR / "buggy_dataset/bugfixes_train.pickle")

# full_file_code_
for row in df.iterrows():
    # bug part
    code: str = row[1]["full_file_code_before_merge"]
    ast = get_tree_from_code(code.encode())
    ast_json = nx.node_link_data(ast)
    with open(TRAIN_DIR / f"{counter}.json", "w") as f:
        json.dump({"code": code, "ast": ast_json}, f)
    data, maps = nx_to_pyg_graph(ast)
    with open(TRAIN_DIR / f"{counter}.pkl", "wb") as f:
        pickle.dump(maps, f)
    torch.save((data, 1), TRAIN_DIR / f"batch_{counter}.pt")

    # fixed part
    counter += 1
    code: str = row[1]["full_file_code_after_merge"]
    ast = get_tree_from_code(code.encode())
    ast_json = nx.node_link_data(ast)
    with open(TRAIN_DIR / f"{counter}.json", "w") as f:
        json.dump({"code": code, "ast": ast_json}, f)
    data, maps = nx_to_pyg_graph(ast)
    with open(TRAIN_DIR / f"{counter}.pkl", "wb") as f:
        pickle.dump(maps, f)
    torch.save((data, 0), TRAIN_DIR / f"batch_{counter}.pt")
    counter += 1

# Training and testing the model

In [3]:
import json
from pathlib import Path
from mt.helper import flatten
from datetime import datetime
from dateutil.parser import parse
from typing import Any
import networkx as nx
from torch_geometric.data import Data, Batch
import requests
import re
import torch
from torch.utils.data import Dataset
from torch_geometric.loader import DataLoader
import pickle
import concurrent.futures
import math
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import networkx as nx
import numpy as np

In [4]:
if (path:=Path("type_to_int.json")).exists():
    with open(path) as f:
        node_type_to_int = json.load(f)
else:
    response = requests.get("https://raw.githubusercontent.com/tree-sitter/tree-sitter-python/master/src/node-types.json")
    types = re.findall(r'"type": "(.+)"', response.text)
    node_type_to_int =  {"UNK": 0} | {t: i+1 for i, t in enumerate(list(set(types)))}
    with open("type_to_int.json", "w") as f:
        json.dump(node_type_to_int, f)


edge_type_to_int = {
    "child": 0,
    "occurance_of": 1,
    "may_next_use": 2,
}

In [5]:
class GraphDataset(Dataset):
    def __init__(self, root_dir: Path) -> None:
        self.root_dir = root_dir
        self.files = list(root_dir.glob("*.pt"))
        self.num_samples = len(self.files)

    def __len__(self) -> int:
        return self.num_samples

    def __getitem__(self, idx: int) -> tuple[str, Batch, torch.FloatTensor]:
        data, target = torch.load(self.root_dir / f"batch_{idx}.pt")
        return data, target

In [6]:
# Instantiate the dataset
dataset = GraphDataset(root_dir=TRAIN_DIR)

# Create a DataLoader
dataloader = DataLoader(dataset, batch_size=16, shuffle=True)

In [7]:
import torch
from torch_geometric.data import Data, Batch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import (
    GATv2Conv,
    global_mean_pool,
    AttentionalAggregation,
    LayerNorm,
    GraphMultisetTransformer,
    BatchNorm
)


class NodeFeatureEmbedding(nn.Module):
    def __init__(self, num_embeddings, embedding_dim):
        super(NodeFeatureEmbedding, self).__init__()
        self.embedding = nn.Embedding(num_embeddings, embedding_dim)

    def forward(self, x):
        # Assuming x is of shape (num_nodes, 1) where each entry is the category index
        return self.embedding(
            x.squeeze()
        )  # Embedding lookup and squeeze to drop extra dimension


class EdgeFeatureEmbedding(nn.Module):
    def __init__(self, num_embeddings, embedding_dim):
        super(EdgeFeatureEmbedding, self).__init__()
        self.embedding = nn.Embedding(num_embeddings, embedding_dim)

    def forward(self, edge_attr):
        # Assuming edge_attr is of shape (num_edges, 1) where each entry is the category index
        return self.embedding(
            edge_attr.squeeze()
        )  # Embedding lookup and squeeze to drop extra dimension


class GATModel(nn.Module):
    def __init__(self, node_embedding_dim, edge_embedding_dim, hidden_channels, out_channels, num_heads=2, num_clusters=10):
        super(GATModel, self).__init__()
        # Layers setup
        self.conv1 = GATv2Conv(node_embedding_dim, hidden_channels, heads=num_heads, concat=True, edge_dim=edge_embedding_dim, dropout=0.06)
        self.bn1 = BatchNorm(hidden_channels * num_heads)
        
        self.conv2 = GATv2Conv(hidden_channels * num_heads, hidden_channels, heads=num_heads, concat=True, edge_dim=edge_embedding_dim, dropout=0.06)
        self.bn2 = BatchNorm(hidden_channels * num_heads)

        self.conv3 = GATv2Conv(hidden_channels * num_heads, hidden_channels, heads=1, concat=False, edge_dim=edge_embedding_dim, dropout=0.06)
        self.bn3 = BatchNorm(hidden_channels)

        # Residual connections
        self.skip1 = nn.Linear(node_embedding_dim, hidden_channels * num_heads)
        self.skip2 = nn.Linear(hidden_channels * num_heads, hidden_channels * num_heads)

        # Pooling and classification
        self.pool = GraphMultisetTransformer(channels=hidden_channels, k=num_clusters, num_encoder_blocks=2, heads=num_heads, layer_norm=True, dropout=0.1)
        self.classifier = nn.Sequential(
            nn.Linear(hidden_channels, hidden_channels),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(hidden_channels, hidden_channels),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(hidden_channels, out_channels)
        )

    def forward(self, x, edge_index, edge_attr, batch):
        # Initial feature transformations
        res1 = F.relu(self.skip1(x))
        x = F.relu(self.bn1(self.conv1(x, edge_index, edge_attr))) + res1

        res2 = F.relu(self.skip2(res1))
        x = F.relu(self.bn2(self.conv2(x, edge_index, edge_attr))) + res2

        x = F.relu(self.bn3(self.conv3(x, edge_index, edge_attr)))

        # Pooling to leverage graph structure
        x = self.pool(x, batch)
        
        # Final classification
        return torch.sigmoid(self.classifier(x))

In [8]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [9]:
node_embedding_dim = 64
edge_embedding_dim = 32
num_node_features = len(node_type_to_int)
num_edge_features = len(edge_type_to_int)

node_embedding_module = NodeFeatureEmbedding(num_embeddings=num_node_features, embedding_dim=node_embedding_dim)
edge_embedding_module = EdgeFeatureEmbedding(num_embeddings=num_edge_features, embedding_dim=edge_embedding_dim)
model = GATModel(node_embedding_dim=node_embedding_dim, edge_embedding_dim=edge_embedding_dim, hidden_channels=64, out_channels=1)
model.to(device)
edge_embedding_module.to(device)
node_embedding_module.to(device)

NodeFeatureEmbedding(
  (embedding): Embedding(216, 64)
)

In [10]:
optimizer = torch.optim.Adam(
    list(edge_embedding_module.parameters())
    + list(node_embedding_module.parameters())
    + list(model.parameters()),
    lr=0.001,
)
criterion = nn.BCEWithLogitsLoss()

def train():
    total_loss = 0
    for graph_batch, targets in dataloader:
        # Transfer data to the device
        x = graph_batch.x.to(device)
        edge_index = graph_batch.edge_index.to(device)
        edge_attr = graph_batch.edge_attr.to(device)
        batch = graph_batch.batch.to(device)
        targets = targets.to(device).float()
        
        optimizer.zero_grad()
        
        # Embedding step
        embedded_edge_attr = edge_embedding_module(edge_attr)
        embedded_x = node_embedding_module(x)
        
        # Model prediction
        prediction = model(embedded_x, edge_index, embedded_edge_attr, batch)
        # print(prediction)
        # print(targets, prediction)
        loss = criterion(prediction.squeeze(), targets)
        
        loss.backward()
        optimizer.step()
        
        total_loss += loss.item()

    epoch_loss = total_loss / len(dataloader)
    return epoch_loss

In [11]:
weight_path = Path("best_weights_dataset.pth")
best_loss = float('inf')  # Initialize best_loss to infinity
best_model_state = None   # To store the state of the best model

for train_loop in range(5):
    print(f"{datetime.now()}: Training loop {train_loop + 1}")
    for param_group in optimizer.param_groups:
        param_group["lr"] *= 0.01

    if weight_path.exists():
        weights = torch.load(weight_path)
        node_emb_weights, edge_emb_weights, model_weights = (
            weights["node_embedding_module_state_dict"],
            weights["edge_embedding_module_state_dict"],
            weights["model_state_dict"],
        )
        node_embedding_module.load_state_dict(node_emb_weights)
        edge_embedding_module.load_state_dict(edge_emb_weights)
        model.load_state_dict(model_weights)

    model.train()
    node_embedding_module.train()
    edge_embedding_module.train()

    best_loss = float("inf")  # Initialize best_loss to infinity
    best_model_state = None  # To store the state of the best model

    for epoch in range(20):  # For example, 10 epochs
        epoch_loss = train()
        print(f"{datetime.now()}: Epoch {epoch+1}: Loss = {epoch_loss}")

        if epoch_loss < best_loss:
            best_loss = epoch_loss
            # Save the model state
            best_model_state = {
                "epoch": epoch,
                "model_state_dict": model.state_dict(),
                "node_embedding_module_state_dict": node_embedding_module.state_dict(),
                "edge_embedding_module_state_dict": edge_embedding_module.state_dict(),
                "optimizer_state_dict": optimizer.state_dict(),
                "loss": best_loss,
            }
            # Example saving to file, adjust the path as necessary
            torch.save(best_model_state, weight_path)

    print(f"{datetime.now()}: Best Loss: {best_loss}")

2024-05-13 08:41:52.239696: Training loop 1
2024-05-13 08:45:14.997766: Epoch 1: Loss = 0.7117978289512332
2024-05-13 08:48:44.920034: Epoch 2: Loss = 0.6990200039685279
2024-05-13 08:52:20.998079: Epoch 3: Loss = 0.6956672842711971
2024-05-13 08:55:53.195897: Epoch 4: Loss = 0.6947498374890335
2024-05-13 08:59:20.849427: Epoch 5: Loss = 0.6941202185309304
2024-05-13 09:02:56.627385: Epoch 6: Loss = 0.693884882035404
2024-05-13 09:06:33.213996: Epoch 7: Loss = 0.6933198553306861
2024-05-13 09:09:58.934862: Epoch 8: Loss = 0.6937818030460022


KeyboardInterrupt: 

In [52]:
weights = torch.load(weight_path)
node_emb_weights, edge_emb_weights, model_weights = (
    weights["node_embedding_module_state_dict"],
    weights["edge_embedding_module_state_dict"],
    weights["model_state_dict"],
)
node_embedding_module.load_state_dict(node_emb_weights)
edge_embedding_module.load_state_dict(edge_emb_weights)
model.load_state_dict(model_weights)

model.eval()
node_embedding_module.eval()
edge_embedding_module.eval()

total, correct = 0, 0
with torch.no_grad():
    for graph_batch, targets in dataloader:
        # Transfer data to the device
        x = graph_batch.x.to(device)
        edge_index = graph_batch.edge_index.to(device)
        edge_attr = graph_batch.edge_attr.to(device)
        batch = graph_batch.batch.to(device)
        targets = targets.to(device).float()

        # Embedding step
        embedded_edge_attr = edge_embedding_module(edge_attr)
        embedded_x = node_embedding_module(x)
        
        # Model prediction
        prediction = model(embedded_x, edge_index, embedded_edge_attr, batch)
        print(prediction)
        total += targets.size(0)
        correct += (prediction == targets).sum().item()

total, correct

tensor([[-0.0309],
        [-0.0381],
        [-0.0304],
        [-0.0407],
        [-0.0335],
        [-0.0244],
        [-0.0371],
        [-0.0356],
        [-0.0119],
        [-0.0346],
        [-0.0369],
        [-0.0365],
        [-0.0367],
        [-0.0394],
        [-0.0333],
        [-0.0392],
        [-0.0370],
        [-0.0348],
        [-0.0387],
        [-0.0341],
        [-0.0395],
        [-0.0217],
        [ 0.0201],
        [-0.0401],
        [-0.0378],
        [-0.0294],
        [-0.0371],
        [-0.0315],
        [-0.0244],
        [-0.0396],
        [-0.0168],
        [-0.0364],
        [-0.0323],
        [-0.0340],
        [-0.0435],
        [-0.0361],
        [-0.0323],
        [-0.0421],
        [-0.0422],
        [-0.0335],
        [-0.0324],
        [-0.0355],
        [-0.0254],
        [-0.0249],
        [-0.0161],
        [-0.0395],
        [-0.0334],
        [-0.0393],
        [-0.0324],
        [-0.0147],
        [-0.0316],
        [-0.0360],
        [-0.

(28236, 0)

In [ ]:
correct = 0
total = 0
# since we're not training, we don't need to calculate the gradients for our outputs
with torch.no_grad():
    for data, targets in testloader:
        # calculate outputs by running images through the network
        outputs = model(images)
        # the class with the highest energy is what we choose as prediction
        _, predicted = torch.max(outputs.data, 1)
        total += targets.size(0)
        correct += (predicted == targets).sum().item()

print(f'Accuracy of the network on the 10000 test images: {100 * correct // total} %')